# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [9]:
# import libraries
import joblib

import pandas as pd
from sqlalchemy import create_engine
import sklearn
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

nltk.download('wordnet') # download for lemmatization
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\inwo9\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\inwo9\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\inwo9\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
# load data from database
engine = create_engine('sqlite:///DisasterResponse.db')
df = pd.read_sql('DisasterResponse', con=engine)
X = df['message']
Y = df.drop(columns=['id', 'message', 'original', 'genre'])
print(X.head(5))
print(Y.head(5))

0    Weather update - a cold front from Cuba that c...
1              Is the Hurricane over or is it not over
2                      Looking for someone but no name
3    UN reports Leogane 80-90 destroyed. Only Hospi...
4    says: west side of Haiti, rest of the country ...
Name: message, dtype: object
   related  request  offer  aid_related  medical_help  medical_products  \
0        1        0      0            0             0                 0   
1        1        0      0            1             0                 0   
2        1        0      0            0             0                 0   
3        1        1      0            1             0                 1   
4        1        0      0            0             0                 0   

   search_and_rescue  security  military  child_alone  ...  aid_centers  \
0                  0         0         0            0  ...            0   
1                  0         0         0            0  ...            0   
2                  0

### 2. Write a tokenization function to process your text data

In [3]:
def tokenize(text):
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()
    tokens = [t for t in tokens if t not in stopwords.words("english")]

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [4]:
pipeline = Pipeline([
        ('vect', TfidfVectorizer(tokenizer=tokenize)),
        ('clf', MultiOutputClassifier(RandomForestClassifier()))
    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [5]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y)

pipeline.fit(X_train, Y_train)

Y_pred = pipeline.predict(X_test)

C:\Users\inwo9\miniconda3\envs\ds_nano\Lib\site-packages\sklearn\feature_extraction\text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [6]:
Y_pred_test = pipeline.predict(X_test)

In [7]:
for i, col in enumerate(Y.columns.values):
    print(col)
    print(classification_report(Y_test[col].values, Y_pred_test[:, i]))
    print('-------------------------------------------------------')

related
              precision    recall  f1-score   support

           0       0.76      0.36      0.49      1528
           1       0.82      0.96      0.89      4979
           2       0.50      0.04      0.08        47

    accuracy                           0.82      6554
   macro avg       0.69      0.46      0.49      6554
weighted avg       0.81      0.82      0.79      6554

-------------------------------------------------------
request
              precision    recall  f1-score   support

           0       0.90      0.98      0.94      5436
           1       0.86      0.48      0.62      1118

    accuracy                           0.90      6554
   macro avg       0.88      0.73      0.78      6554
weighted avg       0.90      0.90      0.89      6554

-------------------------------------------------------
offer
              precision    recall  f1-score   support

           0       0.99      1.00      1.00      6520
           1       0.00      0.00      0.00      

C:\Users\inwo9\miniconda3\envs\ds_nano\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\inwo9\miniconda3\envs\ds_nano\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\inwo9\miniconda3\envs\ds_nano\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result)

### 6. Improve your model
Use grid search to find better parameters. 

In [19]:
parameters = {'clf__estimator__max_depth': [2, 3],
              'clf__estimator__n_estimators': [10, 50]}

cv = GridSearchCV(pipeline, param_grid=parameters, scoring='f1_micro')

cv.fit(X_train, Y_train)

C:\Users\inwo9\miniconda3\envs\ds_nano\Lib\site-packages\sklearn\feature_extraction\text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
C:\Users\inwo9\miniconda3\envs\ds_nano\Lib\site-packages\sklearn\model_selection\_validation.py:982: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\inwo9\miniconda3\envs\ds_nano\Lib\site-packages\sklearn\model_selection\_validation.py", line 971, in _score
    scores = scorer(estimator, X_test, y_test, **score_params)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\inwo9\miniconda3\envs\ds_nano\Lib\site-packages\sklearn\metrics\_scorer.py", line 279, in __call__
    return self._score(partial(_cached_call, None), estimator, X, y_true, **_kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  

GridSearchCV(estimator=Pipeline(steps=[('vect',
                                        TfidfVectorizer(tokenizer=<function tokenize at 0x000001D79A8BFD80>)),
                                       ('clf',
                                        MultiOutputClassifier(estimator=RandomForestClassifier()))]),
             param_grid={'clf__estimator__max_depth': [2, 3],
                         'clf__estimator__n_estimators': [10, 50]},
             scoring='f1_micro')

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [20]:
cv.best_params_

{'clf__estimator__max_depth': 2, 'clf__estimator__n_estimators': 10}

In [21]:
Y_pred_test = pipeline.predict(X_test)

for i, col in enumerate(Y.columns.values):
    print(col)
    print(classification_report(Y_test[col].values, Y_pred_test[:, i]))
    print('-------------------------------------------------------')

related
              precision    recall  f1-score   support

           0       0.76      0.36      0.49      1528
           1       0.82      0.96      0.89      4979
           2       0.50      0.04      0.08        47

    accuracy                           0.82      6554
   macro avg       0.69      0.46      0.49      6554
weighted avg       0.81      0.82      0.79      6554

-------------------------------------------------------
request
              precision    recall  f1-score   support

           0       0.90      0.98      0.94      5436
           1       0.86      0.48      0.62      1118

    accuracy                           0.90      6554
   macro avg       0.88      0.73      0.78      6554
weighted avg       0.90      0.90      0.89      6554

-------------------------------------------------------
offer
              precision    recall  f1-score   support

           0       0.99      1.00      1.00      6520
           1       0.00      0.00      0.00      

C:\Users\inwo9\miniconda3\envs\ds_nano\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\inwo9\miniconda3\envs\ds_nano\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\inwo9\miniconda3\envs\ds_nano\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result)

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [22]:
joblib.dump(pipeline, 'model.pkl') 

['model.pkl']

In [23]:
# check if loading the saved model works
loaded_pipeline = joblib.load('model.pkl')
predictions = loaded_pipeline.predict(X_test)

for i, col in enumerate(Y.columns.values):
    print(col)
    print(classification_report(Y_test[col].values, predictions[:, i]))
    print('-------------------------------------------------------')

related
              precision    recall  f1-score   support

           0       0.76      0.36      0.49      1528
           1       0.82      0.96      0.89      4979
           2       0.50      0.04      0.08        47

    accuracy                           0.82      6554
   macro avg       0.69      0.46      0.49      6554
weighted avg       0.81      0.82      0.79      6554

-------------------------------------------------------
request
              precision    recall  f1-score   support

           0       0.90      0.98      0.94      5436
           1       0.86      0.48      0.62      1118

    accuracy                           0.90      6554
   macro avg       0.88      0.73      0.78      6554
weighted avg       0.90      0.90      0.89      6554

-------------------------------------------------------
offer
              precision    recall  f1-score   support

           0       0.99      1.00      1.00      6520
           1       0.00      0.00      0.00      

C:\Users\inwo9\miniconda3\envs\ds_nano\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\inwo9\miniconda3\envs\ds_nano\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\inwo9\miniconda3\envs\ds_nano\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result)

              precision    recall  f1-score   support

           0       0.88      0.98      0.92      5342
           1       0.80      0.39      0.52      1212

    accuracy                           0.87      6554
   macro avg       0.84      0.68      0.72      6554
weighted avg       0.86      0.87      0.85      6554

-------------------------------------------------------


### 10. Use this notebook to complete `train_classifier.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.